# Scraping all the product information from Stock X

# Imports, Classes and Functions

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', None)

In [2]:
def model_creator(type1,type2):
    if (type(type1) == str) & (type(type2) == str):
        if ('other' not in type1.lower()) & ('other' not in type2.lower()):
            return type1 +' '+ type2
        elif 'other' not in type1.lower():
            return type1
        else:
            return type2
        
    else:
        np.nan

In [66]:
class stock_x_scraper():
    """
    Scraper initialisation, used to scrape product information from Stock X
    Information scraped: 
    - stockx_name : stock x listing name
    - type_1 : first nested category
    - type_2 : second nested category
    - right : first image of the shoe, facing right
    - Style : style code of the shoe
    - Colorway : Colorway of shoe
    - RetailPrice : Price on release
    - ReleaseDate : Date Released
    
    Requires: 
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from bs4 import BeautifulSoup
    from tqdm.notebook import tqdm
    
    """
    def __init__(self, drive=False):
        """
        Argument:
        drive - set to fault by default, opens an instance of chrome which will be automated
        """
        if drive == True:
            self.driver_ = webdriver.Chrome(executable_path='./chromedriver')
        else:
            self.driver_ = None
        
    def driver_on(self):
        """
        Initialises the Selenium Chrome Driver
        """
        self.driver_ = webdriver.Chrome(executable_path='./chromedriver')
    
    def df_set_up(self, df):
        """
        Arguments:
        df: dataframe

        Effect:
        Creates the columns ready for the scrape
        """
        df.loc[:,'stockx_name'] = np.nan
        df.loc[:,'type_1'] = np.nan
        df.loc[:,'type_2'] = np.nan
        df.loc[:,'type_3'] = np.nan
        df.loc[:,'model'] = np.nan
        df.loc[:,'right'] = np.nan
        df.loc[:,'r_d'] = np.nan
        df.loc[:,'straight'] = np.nan
        df.loc[:,'l_d'] = np.nan
        df.loc[:,'left'] = np.nan
        df.loc[:,'back'] = np.nan
        df.loc[:,'Style'] = np.nan
        df.loc[:,'Colorway'] = np.nan
        df.loc[:,'RetailPrice'] = np.nan
        df.loc[:,'ReleaseDate'] = np.nan
    
    
    
    def scraper(self, df, save_name, column_link = 0, column_title = 1, column_type_1 = 2, column_type_2 = 3, 
                column_href = 6, wait_time = 200, start=0, amount=100):
        """
        Arguments: 
        df: dataframe
        save_name: str, what you want the saved file to be called
        column_link: int, column index where the hyperlinks to scrape information are
        column_title: int, column index where the titles will be stored
        column_type_1: int, column index where the type_1 will be stored
        column_type_2: int, column index where the type_2 will be stored
        column_href: int, column index where the img01 of the shoe will be stored
        wait_time: int/float, how long each page will wait on
        start: int, where the scraping will start from
        amount: int, rows to scrape to
        """
        
        for n in tqdm(range(start, amount)):
            # Retrieves the link from the dataframe
            self.driver_.get(df.iloc[n,column_link])
            self.driver_.implicitly_wait(wait_time)

            # Retrieves the stockx title
            for title in self.driver_.find_elements_by_class_name('name'):
                df.iloc[n,column_title] = title.text

            # Retrieves the first image link
            images = self.driver_.find_elements_by_tag_name('img')
            for image in images:
                if 'nike' in image.get_attribute('src').lower():
                    df.iloc[n, column_href] = image.get_attribute('src')

            # Obtains the source of the page
            tutorial_soup = BeautifulSoup(self.driver_.page_source, 'html.parser')

            # The nested categories show up at the 4th and 5th position
            counter = 1
            for typ in tutorial_soup.find('ul', attrs={'class':'breadcrumb'}):
                if counter == 4:
                    df.iloc[n,column_type_1] = typ.text
                elif counter == 5:
                    df.iloc[n,column_type_2] = typ.text
                counter+=1
    

            # Retrieves, style, colorway, release date, retail
            for detail in tutorial_soup.find_all('div',attrs={'class':'detail'}):
            
                if ('Retail' in detail.text.split()) | ('Release' in detail.text.split()):
                    df.loc[n,(''.join(detail.text.split()[:2]))] = (''.join(detail.text.split()[2:]))
                else:
                    df.loc[n,(detail.text.split()[0])]  = (' '.join(detail.text.split()[1:]))
            



In [16]:
!ls

chromedriver                         stockx_product_scrape_complete.ipynb
scrape5.0.csv                        stockx_v2.csv
stockx_link.csv                      test.csv
stockx_nike_scrape.ipynb             test_2.csv
stockx_product_scrape.ipynb


In [17]:
stockx = pd.read_csv('stockx_v2.csv',index_col=0)

In [21]:
stockx.head()

,stockx_link
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white
3,https://stockx.com/nike-dunk-low-sp-syracuse
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020


In [18]:
stockx.tail()

,stockx_link
941,https://stockx.com/nike-zoom-freak-1-black-white
942,https://stockx.com/nike-air-tech-challenge-2-hot-lava-2019
943,https://stockx.com/nike-lebron-16-low-triple-black
944,https://stockx.com/nike-kd-4-think-16-thunderstruck
945,https://stockx.com/nike-sb-dunk-low-east-west-pack-biggie


# Scraping the first 10 as a test

In [67]:
scraper = stock_x_scraper()

In [68]:
scraper.driver_

In [165]:
scraper.driver_on()

In [31]:
scraper.df_set_up(stockx)

In [33]:
stockx.head()

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://stockx.com/nike-dunk-low-sp-syracuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
scraper.scraper(stockx, 'test', amount=10)

In [44]:
stockx.head(10)

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon,Nike LD Waffle Sacai White Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-White-Nylon/Images/Nike-LD-Waffle-Sacai-White-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584025585",NaN,NaN,NaN,NaN,NaN,BV0073-101,White/White,$160,03/10/2020
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon,Nike LD Waffle Sacai Black Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-Black-Nylon/Images/Nike-LD-Waffle-Sacai-Black-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584757351",NaN,NaN,NaN,NaN,NaN,BV0073-002,Black/Black,$160,03/10/2020
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
3,https://stockx.com/nike-dunk-low-sp-syracuse,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020,Nike Air Max 90 Reverse Duck Camo (2020),Air Max,90,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-90-Reverse-Duck-Camo-2020.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1584067533",NaN,NaN,NaN,NaN,NaN,CW6024-600,Infrared/Black-Dark Sage-Baroque Brown,$140,03/26/2020
5,https://stockx.com/nike-dunk-low-sp-kentucky,Nike Dunk Low SP Kentucky (2020),Basketball,Dunk,NaN,NaN,"https://stockx-360.imgix.net/Nike-Dunk-Low-SP-Kentucky/Images/Nike-Dunk-Low-SP-Kentucky/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583212362",NaN,NaN,NaN,NaN,NaN,CU1726-100,White/Varsity Royal,$100,03/14/2020
6,https://stockx.com/nike-air-max-1-london,Nike Air Max 1 London,Air Max,1,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-1-London.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1583909335",NaN,NaN,NaN,NaN,NaN,CV1639-001,Smoke Grey/Valerian Blue-Dark Smoke Grey,$160,03/19/2020
7,https://stockx.com/nike-sb-dunk-low-travis-scott,Nike SB Dunk Low Travis Scott (Regular Box),SB,Nike SB Dunk Low Travis Scott (Regular Box),NaN,NaN,"https://stockx-360.imgix.net/Nike-SB-Dunk-Low-Travis-Scott/Images/Nike-SB-Dunk-Low-Travis-Scott/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1582235884",NaN,NaN,NaN,NaN,NaN,CT5053-001,Black/Black-Parachute Beige-Petra Brown,$150,02/29/2020
8,https://stockx.com/nike-sb-dunk-low-atmos-safari,Nike SB Dunk Low Safari,SB,SB Dunk Low,NaN,NaN,"https://stockx-360.imgix.net/Nike-SB-Dunk-Low-Safari/Images/Nike-SB-Dunk-Low-Safari/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1580325806",NaN,NaN,NaN,NaN,NaN,CD2563-002,Black/Orange-Cement Grey-Outdoor green,$100,03/14/2020
9,https://stockx.com/nike-air-max-1-amsterdam,Nike Air Max 1 Amsterdam,Air Max,1,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-1-Amsterdam.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1583909333",NaN,NaN,NaN,NaN,NaN,CV1638-200,Red Bark/Khaki-Terra Blush-Dusty Peach,$160,03/19/2020


In [48]:
!ls

chromedriver                         stockx_product_scrape_complete.ipynb
scrape5.0.csv                        stockx_v2.csv
stockx_link.csv                      test.csv
stockx_nike_scrape.ipynb             test_2.csv
stockx_product_scrape.ipynb


In [49]:
stockx.to_csv('scrape5.csv')

In [140]:
test_7 = pd.read_csv('scrape5.csv',index_col=0)

In [128]:
for col, angle in [('r_d','img04'),('straight','img10'),('l_d','img13'),('left','img19'),('back','img28')]:
    test_7[col] = test_7.right.apply(lambda x: x.replace('img01',angle))
    

In [129]:
test_7['type_3'] = test_7.apply(lambda x: model_creator(x[2],x[3]),axis = 1)

In [130]:
test_7.stockx_name.apply(lambda x: ' '.join(x.split()[1:4]))

0    LD Waffle Sacai
1    LD Waffle Sacai
2        Air Force 1
3        Air Force 1
4         Air Max 90
5        Dunk Low SP
6          Air Max 1
7        SB Dunk Low
8        SB Dunk Low
9          Air Max 1
Name: stockx_name, dtype: object

In [131]:
test_7['model'] = test_7.stockx_name.apply(lambda x: ' '.join(x.split()[1:4]))

In [133]:
test_7.to_csv('test_7_full.csv')

In [136]:
test_7_full = pd.read_csv('test_7_full.csv', index_col=0)

In [114]:
def path_to_image_html(path):
    """
    Allows html image links to be seen within pandas
    """
    return '<img src="'+ path + '"/>'

In [137]:
HTML(test_7_full.to_html(escape=False ,formatters=dict(right = path_to_image_html, r_d = path_to_image_html,
                                                 straight = path_to_image_html, l_d = path_to_image_html,
                                                 left = path_to_image_html, back = path_to_image_html)))

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon,Nike LD Waffle Sacai White Nylon,Other,Running,Running,LD Waffle Sacai,,,,,,,BV0073-101,White/White,$160,03/10/2020
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon,Nike LD Waffle Sacai Black Nylon,Other,Running,Running,LD Waffle Sacai,,,,,,,BV0073-002,Black/Black,$160,03/10/2020
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white,Nike Air Force 1 Low Supreme White,Air Force,1,Air Force 1,Air Force 1,,,,,,,CU9225-100,White/White-White,$96,03/05/2020
3,https://stockx.com/nike-dunk-low-sp-syracuse,Nike Air Force 1 Low Supreme White,Air Force,1,Air Force 1,Air Force 1,,,,,,,CU9225-100,White/White-White,$96,03/05/2020
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020,Nike Air Max 90 Reverse Duck Camo (2020),Air Max,90,Air Max 90,Air Max 90,,,,,,,CW6024-600,Infrared/Black-Dark Sage-Baroque Brown,$140,03/26/2020
5,https://stockx.com/nike-dunk-low-sp-kentucky,Nike Dunk Low SP Kentucky (2020),Basketball,Dunk,Basketball Dunk,Dunk Low SP,,,,,,,CU1726-100,White/Varsity Royal,$100,03/14/2020
6,https://stockx.com/nike-air-max-1-london,Nike Air Max 1 London,Air Max,1,Air Max 1,Air Max 1,,,,,,,CV1639-001,Smoke Grey/Valerian Blue-Dark Smoke Grey,$160,03/19/2020
7,https://stockx.com/nike-sb-dunk-low-travis-scott,Nike SB Dunk Low Travis Scott (Regular Box),SB,Nike SB Dunk Low Travis Scott (Regular Box),SB Nike SB Dunk Low Travis Scott (Regular Box),SB Dunk Low,,,,,,,CT5053-001,Black/Black-Parachute Beige-Petra Brown,$150,02/29/2020
8,https://stockx.com/nike-sb-dunk-low-atmos-safari,Nike SB Dunk Low Safari,SB,SB Dunk Low,SB SB Dunk Low,SB Dunk Low,,,,,,,CD2563-002,Black/Orange-Cement Grey-Outdoor green,$100,03/14/2020
9,https://stockx.com/nike-air-max-1-amsterdam,Nike Air Max 1 Amsterdam,Air Max,1,Air Max 1,Air Max 1,,,,,,,CV1638-200,Red Bark/Khaki-Terra Blush-Dusty Peach,$160,03/19/2020


## Main Scrape

In [142]:
stockx_v2 = pd.read_csv('scrape5.csv', index_col=0)

In [145]:
stockx_v2.head(11)

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon,Nike LD Waffle Sacai White Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-White-Nylon/Images/Nike-LD-Waffle-Sacai-White-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584025585",NaN,NaN,NaN,NaN,NaN,BV0073-101,White/White,$160,03/10/2020
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon,Nike LD Waffle Sacai Black Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-Black-Nylon/Images/Nike-LD-Waffle-Sacai-Black-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584757351",NaN,NaN,NaN,NaN,NaN,BV0073-002,Black/Black,$160,03/10/2020
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
3,https://stockx.com/nike-dunk-low-sp-syracuse,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020,Nike Air Max 90 Reverse Duck Camo (2020),Air Max,90,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-90-Reverse-Duck-Camo-2020.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1584067533",NaN,NaN,NaN,NaN,NaN,CW6024-600,Infrared/Black-Dark Sage-Baroque Brown,$140,03/26/2020
5,https://stockx.com/nike-dunk-low-sp-kentucky,Nike Dunk Low SP Kentucky (2020),Basketball,Dunk,NaN,NaN,"https://stockx-360.imgix.net/Nike-Dunk-Low-SP-Kentucky/Images/Nike-Dunk-Low-SP-Kentucky/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583212362",NaN,NaN,NaN,NaN,NaN,CU1726-100,White/Varsity Royal,$100,03/14/2020
6,https://stockx.com/nike-air-max-1-london,Nike Air Max 1 London,Air Max,1,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-1-London.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1583909335",NaN,NaN,NaN,NaN,NaN,CV1639-001,Smoke Grey/Valerian Blue-Dark Smoke Grey,$160,03/19/2020
7,https://stockx.com/nike-sb-dunk-low-travis-scott,Nike SB Dunk Low Travis Scott (Regular Box),SB,Nike SB Dunk Low Travis Scott (Regular Box),NaN,NaN,"https://stockx-360.imgix.net/Nike-SB-Dunk-Low-Travis-Scott/Images/Nike-SB-Dunk-Low-Travis-Scott/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1582235884",NaN,NaN,NaN,NaN,NaN,CT5053-001,Black/Black-Parachute Beige-Petra Brown,$150,02/29/2020
8,https://stockx.com/nike-sb-dunk-low-atmos-safari,Nike SB Dunk Low Safari,SB,SB Dunk Low,NaN,NaN,"https://stockx-360.imgix.net/Nike-SB-Dunk-Low-Safari/Images/Nike-SB-Dunk-Low-Safari/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1580325806",NaN,NaN,NaN,NaN,NaN,CD2563-002,Black/Orange-Cement Grey-Outdoor green,$100,03/14/2020
9,https://stockx.com/nike-air-max-1-amsterdam,Nike Air Max 1 Amsterdam,Air Max,1,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-1-Amsterdam.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1583909333",NaN,NaN,NaN,NaN,NaN,CV1638-200,Red Bark/Khaki-Terra Blush-Dusty Peach,$160,03/19/2020


In [146]:
# quick test to ensure it works
scraper.scraper(stockx_v2, 'test2', start=10, amount=13)

In [149]:
stockx_v2.head(14)

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon,Nike LD Waffle Sacai White Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-White-Nylon/Images/Nike-LD-Waffle-Sacai-White-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584025585",NaN,NaN,NaN,NaN,NaN,BV0073-101,White/White,$160,03/10/2020
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon,Nike LD Waffle Sacai Black Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-Black-Nylon/Images/Nike-LD-Waffle-Sacai-Black-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584757351",NaN,NaN,NaN,NaN,NaN,BV0073-002,Black/Black,$160,03/10/2020
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
3,https://stockx.com/nike-dunk-low-sp-syracuse,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020,Nike Air Max 90 Reverse Duck Camo (2020),Air Max,90,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-90-Reverse-Duck-Camo-2020.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1584067533",NaN,NaN,NaN,NaN,NaN,CW6024-600,Infrared/Black-Dark Sage-Baroque Brown,$140,03/26/2020
5,https://stockx.com/nike-dunk-low-sp-kentucky,Nike Dunk Low SP Kentucky (2020),Basketball,Dunk,NaN,NaN,"https://stockx-360.imgix.net/Nike-Dunk-Low-SP-Kentucky/Images/Nike-Dunk-Low-SP-Kentucky/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583212362",NaN,NaN,NaN,NaN,NaN,CU1726-100,White/Varsity Royal,$100,03/14/2020
6,https://stockx.com/nike-air-max-1-london,Nike Air Max 1 London,Air Max,1,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-1-London.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1583909335",NaN,NaN,NaN,NaN,NaN,CV1639-001,Smoke Grey/Valerian Blue-Dark Smoke Grey,$160,03/19/2020
7,https://stockx.com/nike-sb-dunk-low-travis-scott,Nike SB Dunk Low Travis Scott (Regular Box),SB,Nike SB Dunk Low Travis Scott (Regular Box),NaN,NaN,"https://stockx-360.imgix.net/Nike-SB-Dunk-Low-Travis-Scott/Images/Nike-SB-Dunk-Low-Travis-Scott/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1582235884",NaN,NaN,NaN,NaN,NaN,CT5053-001,Black/Black-Parachute Beige-Petra Brown,$150,02/29/2020
8,https://stockx.com/nike-sb-dunk-low-atmos-safari,Nike SB Dunk Low Safari,SB,SB Dunk Low,NaN,NaN,"https://stockx-360.imgix.net/Nike-SB-Dunk-Low-Safari/Images/Nike-SB-Dunk-Low-Safari/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1580325806",NaN,NaN,NaN,NaN,NaN,CD2563-002,Black/Orange-Cement Grey-Outdoor green,$100,03/14/2020
9,https://stockx.com/nike-air-max-1-amsterdam,Nike Air Max 1 Amsterdam,Air Max,1,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-1-Amsterdam.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1583909333",NaN,NaN,NaN,NaN,NaN,CV1638-200,Red Bark/Khaki-Terra Blush-Dusty Peach,$160,03/19/2020


In [150]:
stockx_v2.iloc[13,:]

stockx_link    https://stockx.com/nike-dunk-low-off-white-pine-green
stockx_name                                                      NaN
type_1                                                           NaN
type_2                                                           NaN
type_3                                                           NaN
model                                                            NaN
right                                                            NaN
r_d                                                              NaN
straight                                                         NaN
l_d                                                              NaN
left                                                             NaN
back                                                             NaN
Style                                                            NaN
Colorway                                                         NaN
RetailPrice                       

In [151]:
scraper.scraper(stockx_v2, 'test100', start=13, amount=100)

KeyboardInterrupt: 

In [152]:
# pickd up an error at row 40

# https://stockx.com/news/best-nike-shoes-for-women/ this was the link it picked up
stockx_v2.iloc[38:42,:] 

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
38,https://stockx.com/nike-air-force-1-low-clot-blue-silk,Nike Air Force 1 Low CLOT Blue Silk,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Clot-Blue-Silk/Images/Nike-Air-Force-1-Low-Clot-Blue-Silk/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1573502731",NaN,NaN,NaN,NaN,NaN,CJ5290-400,University Blue/White-Gum,$250,11/11/2019
39,https://stockx.com/nike-react-presto-undercover-black,Nike React Presto Undercover Black,Other,Running,NaN,NaN,"https://stockx.imgix.net/Nike-React-Presto-Undercover-Black.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1579711071",NaN,NaN,NaN,NaN,NaN,CU3459-001,Black/White-Black-Black,$180,01/28/2020
40,https://stockx.com/news/best-nike-shoes-for-women/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,https://stockx.com/nike-daybreak-undercover-black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
stockx_v3 = stockx_v2.drop(index=40)

In [166]:
scraper.scraper(stockx_v3, 'test100', start=40, amount=42)

In [168]:
stockx_v3.iloc[40:43]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
41,https://stockx.com/nike-daybreak-undercover-black,Nike Daybreak Undercover Black,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100",NaN,NaN,NaN,NaN,NaN,CJ6107-100,Sail/Navy,$120,08/13/2019
42,https://stockx.com/nike-classic-cortez-stranger-things-sail,Nike Classic Cortez Stranger Things Sail Upside Down Pack,Other,Running,NaN,NaN,"https://stockx.imgix.net/Nike-Classic-Cortez-Stranger-Things-Sail.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1564585836",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,https://stockx.com/nike-air-force-1-low-retro-ivory-snake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
scraper.scraper(stockx_v3, 'test100', start=43, amount=100)

In [173]:
scraper.scraper(stockx_v3, 'test100', start=100, amount=250)

In [175]:
stockx_v3.iloc[:252]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon,Nike LD Waffle Sacai White Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-White-Nylon/Images/Nike-LD-Waffle-Sacai-White-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584025585",NaN,NaN,NaN,NaN,NaN,BV0073-101,White/White,$160,03/10/2020
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon,Nike LD Waffle Sacai Black Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-Black-Nylon/Images/Nike-LD-Waffle-Sacai-Black-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584757351",NaN,NaN,NaN,NaN,NaN,BV0073-002,Black/Black,$160,03/10/2020
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
3,https://stockx.com/nike-dunk-low-sp-syracuse,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020,Nike Air Max 90 Reverse Duck Camo (2020),Air Max,90,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-90-Reverse-Duck-Camo-2020.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1584067533",NaN,NaN,NaN,NaN,NaN,CW6024-600,Infrared/Black-Dark Sage-Baroque Brown,$140,03/26/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,https://stockx.com/nike-vapor-street-off-white-tour-yellow-w,Nike Vapor Street Off-White Tour Yellow (W),Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-Vapor-Street-Off-White-Tour-Yellow-W/Images/Nike-Vapor-Street-Off-White-Tour-Yellow-W/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1575076215",NaN,NaN,NaN,NaN,NaN,432997-107,Sail/Midnight Navy-Gum Yellow,$90,02/25/2017
249,https://stockx.com/nike-killshot-2-j-crew-sail-midnight-navy,Nike Killshot 2 J Crew Sail Midnight Navy,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Killshot-2-J-Crew-Sail-Midnight-Navy/Images/Nike-Killshot-2-J-Crew-Sail-Midnight-Navy/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1538080256",NaN,NaN,NaN,NaN,NaN,BV4594-700,Bright Citron/Black-Summit White,$160,08/01/2019
250,https://stockx.com/nike-daybreak-undercover-bright-citron,Nike Daybreak Undercover Bright Citron,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Bright-Citron/Images/Nike-Daybreak-Undercover-Bright-Citron/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1561409984",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,https://stockx.com/nike-air-max-270-triple-black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
scraper.scraper(stockx_v3, 'test100', start=250, amount=500)

In [178]:
stockx_v3.iloc[248:250]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
249,https://stockx.com/nike-killshot-2-j-crew-sail-midnight-navy,Nike Killshot 2 J Crew Sail Midnight Navy,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Killshot-2-J-Crew-Sail-Midnight-Navy/Images/Nike-Killshot-2-J-Crew-Sail-Midnight-Navy/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1538080256",NaN,NaN,NaN,NaN,NaN,BV4594-700,Bright Citron/Black-Summit White,$160,08/01/2019
250,https://stockx.com/nike-daybreak-undercover-bright-citron,Nike Daybreak Undercover Bright Citron,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Bright-Citron/Images/Nike-Daybreak-Undercover-Bright-Citron/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1561409984",NaN,NaN,NaN,NaN,NaN,AH8050-005,Black/Black-Black,$150,03/16/2018


In [179]:
stockx_v3.shape

(946, 16)

In [180]:
scraper.scraper(stockx_v3, 'test_fin', start=500, amount=947)

InvalidArgumentException: Message: invalid argument: missing command parameters


In [190]:
# RETRIEVED BY HAND, AS THE SITE WAS SET UP IN A DIFFERENT MANNER
stockx_v3.iloc[943:944,:]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
944,https://stockx.com/nike-kd-4-think-16-thunderstruck,Nike KD 4 Think 16 (Thunderstruck),KD,4,NaN,NaN,"https://stockx-360.imgix.net/Nike-KD-4-Think-16-Thunderstruck/Images/Nike-KD-4-Think-16-Thunderstruck/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1538080256",NaN,NaN,NaN,NaN,NaN,883232-441,Binary Blue/White-Gum Light Brown,$100,05/12/2018


In [192]:
stockx_v3.iloc[944,12] = '883232-441'
stockx_v3.iloc[944,13] = 'BINARY BLUE/WHITE-GUM LIGHT BROWN'
stockx_v3.iloc[944,14]= "$100"
stockx_v3.iloc[944,15] = '05/12/2018'

In [193]:
stockx_v3.iloc[944]

stockx_link                                                                                                                                      https://stockx.com/nike-sb-dunk-low-east-west-pack-biggie
stockx_name                                                                                                                                                            Nike SB Dunk Low Rivals Pack (East)
type_1                                                                                                                                                                                                  SB
type_2                                                                                                                                                                                         SB Dunk Low
type_3                                                                                                                                                                                      

In [196]:
stockx_v4 = stockx_v3.iloc[:-1,:]

In [197]:
stockx_v4.to_csv('stockx_v4.csv')

## Dataset clean up

In [ ]:
# check for missing values

In [3]:
!ls

chromedriver                         stockx_product_scrape_complete.ipynb
scrape125.0.csv                      stockx_v2.csv
scrape250.0.csv                      stockx_v4.csv
scrape5.0.csv                        test.csv
scrape5.csv                          test_2.csv
scrape50.0.csv                       test_6.csv
stockx_link.csv                      test_7.csv
stockx_nike_scrape.ipynb             test_7_full.csv
stockx_product_scrape.ipynb


In [29]:
stockx_v5 = pd.read_csv('stockx_v4.csv',index_col=0)

In [30]:
stockx_v5

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
0,https://stockx.com/nike-ld-waffle-sacai-white-nylon,Nike LD Waffle Sacai White Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-White-Nylon/Images/Nike-LD-Waffle-Sacai-White-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584025585",NaN,NaN,NaN,NaN,NaN,BV0073-101,White/White,$160,03/10/2020
1,https://stockx.com/nike-ld-waffle-sacai-black-nylon,Nike LD Waffle Sacai Black Nylon,Other,Running,NaN,NaN,"https://stockx-360.imgix.net/Nike-LD-Waffle-Sacai-Black-Nylon/Images/Nike-LD-Waffle-Sacai-Black-Nylon/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1584757351",NaN,NaN,NaN,NaN,NaN,BV0073-002,Black/Black,$160,03/10/2020
2,https://stockx.com/nike-air-force-1-low-supreme-box-logo-white,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
3,https://stockx.com/nike-dunk-low-sp-syracuse,Nike Air Force 1 Low Supreme White,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Images/Nike-Air-Force-1-Low-Supreme-Box-Logo-White/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1583951149",NaN,NaN,NaN,NaN,NaN,CU9225-100,White/White-White,$96,03/05/2020
4,https://stockx.com/nike-air-max-90-reverse-duck-camo-2020,Nike Air Max 90 Reverse Duck Camo (2020),Air Max,90,NaN,NaN,"https://stockx.imgix.net/Nike-Air-Max-90-Reverse-Duck-Camo-2020.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1584067533",NaN,NaN,NaN,NaN,NaN,CW6024-600,Infrared/Black-Dark Sage-Baroque Brown,$140,03/26/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,https://stockx.com/nike-zoom-freak-1-black-white,Nike Zoom Freak 1 Black White,Basketball,Other,NaN,NaN,"https://stockx.imgix.net/Nike-Zoom-Freak-1-Black-White.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1561565512",NaN,NaN,NaN,NaN,NaN,CJ1437-100,White/Hot Lava-Black-Granite,$130,05/24/2019
942,https://stockx.com/nike-air-tech-challenge-2-hot-lava-2019,Nike Air Tech Challenge 2 Hot Lava (2019),Other,Training,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Tech-Challenge-2-Hot-Lava-2019/Images/Nike-Air-Tech-Challenge-2-Hot-Lava-2019/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1560200518",NaN,NaN,NaN,NaN,NaN,CI2668-002,Black/Black-Black,$160,05/13/2019
943,https://stockx.com/nike-lebron-16-low-triple-black,Nike LeBron 16 Low Triple Black,LeBron,16,NaN,NaN,"https://stockx-360.imgix.net/Nike-LeBron-16-Low-Triple-Black/Images/Nike-LeBron-16-Low-Triple-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559252270",NaN,NaN,NaN,NaN,NaN,AQ5103-100,White/Photo Blue-Wolf Grey-Black,$140,06/07/2018
944,https://stockx.com/nike-kd-4-think-16-thunderstruck,Nike KD 4 Think 16 (Thunderstruck),KD,4,NaN,NaN,"https://stockx-360.imgix.net/Nike-KD-4-Think-16-Thunderstruck/Images/Nike-KD-4-Think-16-Thunderstruck/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1538080256",NaN,NaN,NaN,NaN,NaN,883232-441,Binary Blue/White-Gum Light Brown,$100,05/12/2018


In [ ]:
# I have discovered that the style/color/retailprice/releasedate column is off by 1,
# This is becaues of the error i encountered at index 40. 
# I will use the shift function to push the rows down one and then reassign it in place

In [34]:
stockx_v5.iloc[38:45,:]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
38,https://stockx.com/nike-air-force-1-low-clot-blue-silk,Nike Air Force 1 Low CLOT Blue Silk,Air Force,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Clot-Blue-Silk/Images/Nike-Air-Force-1-Low-Clot-Blue-Silk/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1573502731",NaN,NaN,NaN,NaN,NaN,CJ5290-400,University Blue/White-Gum,$250,11/11/2019
39,https://stockx.com/nike-react-presto-undercover-black,Nike React Presto Undercover Black,Other,Running,NaN,NaN,"https://stockx.imgix.net/Nike-React-Presto-Undercover-Black.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1579711071",NaN,NaN,NaN,NaN,NaN,CU3459-001,Black/White-Black-Black,$180,01/28/2020
41,https://stockx.com/nike-daybreak-undercover-black,Nike Daybreak Undercover Black,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100",NaN,NaN,NaN,NaN,NaN,CJ6107-100,Sail/Navy,$120,08/13/2019
42,https://stockx.com/nike-classic-cortez-stranger-things-sail,Nike Classic Cortez Stranger Things Sail Upside Down Pack,Other,Running,NaN,NaN,"https://stockx.imgix.net/Nike-Classic-Cortez-Stranger-Things-Sail.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1564585836",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,https://stockx.com/nike-air-force-1-low-retro-ivory-snake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CK6643-100,Multi-Color/Multi-Color,$170,04/12/2019
44,https://stockx.com/nike-air-max-1-susan-missing-link,Nike Air Max 1 Susan Missing Link,Air Max,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Max-1-Susan-Missing-Link/Images/Nike-Air-Max-1-Susan-Missing-Link/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1555622494",NaN,NaN,NaN,NaN,NaN,AT8086-002,Black/Black-Fossil,$170,05/17/2019
45,https://stockx.com/nike-air-fear-of-god-moccasin-black,Nike Air Fear Of God Moccasin Black,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Fear-Of-God-Moccasin-Black/Images/Nike-Air-Fear-Of-God-Moccasin-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559162768",NaN,NaN,NaN,NaN,NaN,AO4606-001,Black/White-Cone-Black,$170,12/19/2018


In [39]:
# I need to insert this back into the dataframe at index 40
fixed_columns = stockx_v5.iloc[40:,-4:].shift()

In [44]:
fixed_columns.head()

,Style,Colorway,RetailPrice,ReleaseDate
41,NaN,NaN,NaN,NaN
42,CJ6107-100,Sail/Navy,$120,08/13/2019
43,NaN,NaN,NaN,NaN
44,CK6643-100,Multi-Color/Multi-Color,$170,04/12/2019
45,AT8086-002,Black/Black-Fossil,$170,05/17/2019


In [48]:
stockx_v5.iloc[40:,].head()

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
41,https://stockx.com/nike-daybreak-undercover-black,Nike Daybreak Undercover Black,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100",NaN,NaN,NaN,NaN,NaN,CJ6107-100,Sail/Navy,$120,08/13/2019
42,https://stockx.com/nike-classic-cortez-stranger-things-sail,Nike Classic Cortez Stranger Things Sail Upside Down Pack,Other,Running,NaN,NaN,"https://stockx.imgix.net/Nike-Classic-Cortez-Stranger-Things-Sail.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1564585836",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,https://stockx.com/nike-air-force-1-low-retro-ivory-snake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CK6643-100,Multi-Color/Multi-Color,$170,04/12/2019
44,https://stockx.com/nike-air-max-1-susan-missing-link,Nike Air Max 1 Susan Missing Link,Air Max,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Max-1-Susan-Missing-Link/Images/Nike-Air-Max-1-Susan-Missing-Link/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1555622494",NaN,NaN,NaN,NaN,NaN,AT8086-002,Black/Black-Fossil,$170,05/17/2019
45,https://stockx.com/nike-air-fear-of-god-moccasin-black,Nike Air Fear Of God Moccasin Black,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Fear-Of-God-Moccasin-Black/Images/Nike-Air-Fear-Of-God-Moccasin-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559162768",NaN,NaN,NaN,NaN,NaN,AO4606-001,Black/White-Cone-Black,$170,12/19/2018


In [49]:
stockx_v5.iloc[40:,-4:] = fixed_columns

In [50]:
stockx_v5.iloc[40:]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
41,https://stockx.com/nike-daybreak-undercover-black,Nike Daybreak Undercover Black,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,https://stockx.com/nike-classic-cortez-stranger-things-sail,Nike Classic Cortez Stranger Things Sail Upside Down Pack,Other,Running,NaN,NaN,"https://stockx.imgix.net/Nike-Classic-Cortez-Stranger-Things-Sail.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1564585836",NaN,NaN,NaN,NaN,NaN,CJ6107-100,Sail/Navy,$120,08/13/2019
43,https://stockx.com/nike-air-force-1-low-retro-ivory-snake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,https://stockx.com/nike-air-max-1-susan-missing-link,Nike Air Max 1 Susan Missing Link,Air Max,1,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Max-1-Susan-Missing-Link/Images/Nike-Air-Max-1-Susan-Missing-Link/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1555622494",NaN,NaN,NaN,NaN,NaN,CK6643-100,Multi-Color/Multi-Color,$170,04/12/2019
45,https://stockx.com/nike-air-fear-of-god-moccasin-black,Nike Air Fear Of God Moccasin Black,Other,Other,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Fear-Of-God-Moccasin-Black/Images/Nike-Air-Fear-Of-God-Moccasin-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559162768",NaN,NaN,NaN,NaN,NaN,AT8086-002,Black/Black-Fossil,$170,05/17/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,https://stockx.com/nike-zoom-freak-1-black-white,Nike Zoom Freak 1 Black White,Basketball,Other,NaN,NaN,"https://stockx.imgix.net/Nike-Zoom-Freak-1-Black-White.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1561565512",NaN,NaN,NaN,NaN,NaN,BQ5422-001,Black/Black-White-Lucid Green,$130,07/01/2019
942,https://stockx.com/nike-air-tech-challenge-2-hot-lava-2019,Nike Air Tech Challenge 2 Hot Lava (2019),Other,Training,NaN,NaN,"https://stockx-360.imgix.net/Nike-Air-Tech-Challenge-2-Hot-Lava-2019/Images/Nike-Air-Tech-Challenge-2-Hot-Lava-2019/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1560200518",NaN,NaN,NaN,NaN,NaN,CJ1437-100,White/Hot Lava-Black-Granite,$130,05/24/2019
943,https://stockx.com/nike-lebron-16-low-triple-black,Nike LeBron 16 Low Triple Black,LeBron,16,NaN,NaN,"https://stockx-360.imgix.net/Nike-LeBron-16-Low-Triple-Black/Images/Nike-LeBron-16-Low-Triple-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559252270",NaN,NaN,NaN,NaN,NaN,CI2668-002,Black/Black-Black,$160,05/13/2019
944,https://stockx.com/nike-kd-4-think-16-thunderstruck,Nike KD 4 Think 16 (Thunderstruck),KD,4,NaN,NaN,"https://stockx-360.imgix.net/Nike-KD-4-Think-16-Thunderstruck/Images/Nike-KD-4-Think-16-Thunderstruck/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1538080256",NaN,NaN,NaN,NaN,NaN,AQ5103-100,White/Photo Blue-Wolf Grey-Black,$140,06/07/2018


In [51]:
stockx_v5.to_csv('stockx_v5_fixed.csv')

In [94]:
stockx_v6 = pd.read_csv('stockx_v5_fixed.csv',index_col=0)

In [95]:
# Obtaing the type_3, model value, and picture links

In [96]:
stockx_v6['type_3'] = stockx_v6.apply(lambda x: model_creator(x[2],x[3]),axis = 1)

In [97]:
stockx_v6['model'] = stockx_v6.stockx_name.apply(lambda x: ' '.join(x.split()[1:4]) if type(x) == str else x)

In [98]:
for col, angle in [('r_d','img04'),('straight','img10'),('l_d','img13'),('left','img19'),('back','img28')]:
    stockx_v6[col] = stockx_v6.right.apply(lambda x: x.replace('img01',angle) if type(x) == str else np.nan)


### Checking that the columns are correct for the ones that I shifted

In [77]:
stockx_v6.iloc[445:447][['stockx_link','stockx_name','Style','Colorway','RetailPrice','ReleaseDate']]

,stockx_link,stockx_name,Style,Colorway,RetailPrice,ReleaseDate
446,https://stockx.com/nike-air-force-1-vandalized-sail-chrome-yellow-w,Nike Air Force 1 Vandalized Sail Chrome Yellow (W),BV0740-101,Sail/Chrome Yellow-White-Black,$110,10/01/2019
447,https://stockx.com/nike-air-vapormax-moc-2-acronym-light-bone,Nike Air VaporMax Moc 2 Acronym Light Bone,AQ0996-001,Light Bone/Volt-Light Bone,$225,03/24/2018


![](images/spot_check_1.png)
![](images/spot_check_2.png)

In [76]:
stockx_v6.iloc[45:47][['stockx_link','stockx_name','Style','Colorway','RetailPrice','ReleaseDate']]

,stockx_link,stockx_name,Style,Colorway,RetailPrice,ReleaseDate
46,https://stockx.com/nike-air-force-1-low-off-white-black-white,Nike Air Force 1 Low Off-White Black White,AO4606-001,Black/White-Cone-Black,$170,12/19/2018
47,https://stockx.com/nike-zoom-terra-kiger-5-off-white-white-w,Nike Zoom Terra Kiger 5 Off-White White (W),CD8179-100,White/Metallic Silver-Sail-Safety Orange,$180,06/24/2019


![](images/spot_check_3.png)
![](images/spot_check_4.png)

In [78]:
stockx_v6.iloc[99:101][['stockx_link','stockx_name','Style','Colorway','RetailPrice','ReleaseDate']]

,stockx_link,stockx_name,Style,Colorway,RetailPrice,ReleaseDate
100,https://stockx.com/nike-zoom-fly-off-white,Nike Zoom Fly Off-White,AJ4588-100,White/White-Muslin,$170,11/01/2017
101,https://stockx.com/nike-air-force-1-low-3x1-denim-raw-indigo,Nike Air Force 1 Low 3x1 Denim Raw Indigo,905345-402,Raw Indigo/Raw Indigo-Dark Obsidian-Total Orange-Summit White,$130,05/24/2019


![](images/spot_check_5.png)
![](images/spot_check_6.png)

In [81]:
stockx_v6.iloc[855:857][['stockx_link','stockx_name','Style','Colorway','RetailPrice','ReleaseDate']]

,stockx_link,stockx_name,Style,Colorway,RetailPrice,ReleaseDate
856,https://stockx.com/nike-sb-dunk-low-green-hemp,Nike SB Dunk Low Green Hemp,304292-732,Jersey Gold/Bonsai,$65,04/20/2004
857,https://stockx.com/nike-air-force-1-low-07-lx-blueprint-summit-white-w,Nike Air Force 1 Low 07 LX Blueprint Summit White (W),CI3445-100,Summit White/Summit White-University Red,$110,NaN


![](images/spot_check_7.png)
![](images/spot_check_8.png)

### Null Values

In [99]:
stockx_v6.isnull().sum()

stockx_link     0
stockx_name     1
type_1          1
type_2          1
type_3          1
model           1
right           4
r_d             4
straight        4
l_d             4
left            4
back            4
Style           4
Colorway        2
RetailPrice    48
ReleaseDate    71
dtype: int64

In [100]:
stockx_v6[stockx_v6.stockx_name.isnull() == True]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
43,https://stockx.com/nike-air-force-1-low-retro-ivory-snake,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# Retail Price and Release Date have missing values, because the page changed the way it was coded over the years
# I will manually get the info the single missing row

In [102]:
stockx_v6.iloc[42]['stockx_name'] = 'Nike Air Force 1 Low Retro Ivory Snake'
stockx_v6.iloc[42]['type_1'] = 'Air Force'
stockx_v6.iloc[42]['type_2'] = '1'
stockx_v6.iloc[42]['type_3'] = 'Air Force 1'
stockx_v6.iloc[42]['model'] = 'Air Force 1'
stockx_v6.iloc[42]['right'] = 'https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Images/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1559767319&w=1000'
stockx_v6.iloc[42]['r_d'] = 'https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Images/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1559767319&w=1000'.replace('img01','img04')
stockx_v6.iloc[42]['straight'] = 'https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Images/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1559767319&w=1000'.replace('img01','img10')
stockx_v6.iloc[42]['l_d'] = 'https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Images/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1559767319&w=1000'.replace('img01','img13')
stockx_v6.iloc[42]['left'] = 'https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Images/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1559767319&w=1000'.replace('img01','img19')
stockx_v6.iloc[42]['back'] = 'https://stockx-360.imgix.net/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Images/Nike-Air-Force-1-Low-Retro-Ivory-Snake/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1559767319&w=1000'.replace('img01','img28')
stockx_v6.iloc[42]['Style'] = 'AO1635-100'
stockx_v6.iloc[42]['Colorway'] = 'White/Elemental Gold-Dark Hazel-Black'
stockx_v6.iloc[42]['RetailPrice'] = '$150'
stockx_v6.iloc[42]['ReleaseDate'] = '05/22/2019'

In [103]:
stockx_v6.iloc[42]

stockx_link                                                                                                                                 https://stockx.com/nike-air-force-1-low-retro-ivory-snake
stockx_name                                                                                                                                                    Nike Air Force 1 Low Retro Ivory Snake
type_1                                                                                                                                                                                      Air Force
type_2                                                                                                                                                                                              1
type_3                                                                                                                                                                                    Air Force 1
model     

In [104]:
stockx_v6.isnull().sum()

stockx_link     0
stockx_name     0
type_1          0
type_2          0
type_3          0
model           0
right           3
r_d             3
straight        3
l_d             3
left            3
back            3
Style           3
Colorway        1
RetailPrice    47
ReleaseDate    70
dtype: int64

In [105]:
stockx_v6[stockx_v6.Colorway.isnull() == True]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
41,https://stockx.com/nike-daybreak-undercover-black,Nike Daybreak Undercover Black,Other,Other,Other,Daybreak Undercover Black,"https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100","https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img04.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100","https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img10.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100","https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img13.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100","https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img19.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100","https://stockx-360.imgix.net/Nike-Daybreak-Undercover-Black/Images/Nike-Daybreak-Undercover-Black/Lv2/img28.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1559847100",NaN,NaN,NaN,NaN


In [109]:
stockx_v6.iloc[40][0]

'https://stockx.com/nike-daybreak-undercover-black'

In [110]:
stockx_v6.iloc[40]['Style'] = 'BV4594-001'
stockx_v6.iloc[40]['Colorway'] = 'BLACK/WHITE'
stockx_v6.iloc[40]['RetailPrice'] = '$160'
stockx_v6.iloc[40]['ReleaseDate'] = '06/07/2019'

In [112]:
stockx_v6[stockx_v6.right.isnull() == True]
# 

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
158,https://stockx.com/nike-air-max-270-react-travis-scott,Nike Air Max 270 React ENG Travis Scott,Air Max,Other,Air Max,Air Max 270,NaN,NaN,NaN,NaN,NaN,NaN,CT2864-200,Light Cream/Dark Hazel-Mica Green-Starfish,$190,04/10/2020
162,https://stockx.com/nike-air-mag-back-to-the-future-bttf-2016,Nike MAG Back to the Future (2016),Other,Premium,Premium,MAG Back to,NaN,NaN,NaN,NaN,NaN,NaN,H015-MNOTHR-402,Multi-Color/Multi-Color,NaN,10/04/2016
836,https://stockx.com/nike-air-force-1-07-white-black,Nike Air Force 1 '07 White/Black,Air Force,1,Air Force 1,Air Force 1,NaN,NaN,NaN,NaN,NaN,NaN,CD0884-100,White/White-Black,$90,NaN


In [118]:
stockx_v6.iloc[161]['right'] = 'https://stockx-360.imgix.net/mag-kauf_TruView/Images/mag-kauf_TruView/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1538080256&w=1000'
stockx_v6.iloc[161]['r_d'] = 'https://stockx-360.imgix.net/mag-kauf_TruView/Images/mag-kauf_TruView/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1538080256&w=1000'.replace('img01','img04')
stockx_v6.iloc[161]['straight'] = 'https://stockx-360.imgix.net/mag-kauf_TruView/Images/mag-kauf_TruView/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1538080256&w=1000'.replace('img01','img10')
stockx_v6.iloc[161]['l_d'] = 'https://stockx-360.imgix.net/mag-kauf_TruView/Images/mag-kauf_TruView/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1538080256&w=1000'.replace('img01','img13')
stockx_v6.iloc[161]['left'] = 'https://stockx-360.imgix.net/mag-kauf_TruView/Images/mag-kauf_TruView/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1538080256&w=1000'.replace('img01','img19')
stockx_v6.iloc[161]['back'] = 'https://stockx-360.imgix.net/mag-kauf_TruView/Images/mag-kauf_TruView/Lv2/img01.jpg?auto=format,compress&q=90&updated_at=1538080256&w=1000'.replace('img01','img28')

In [121]:
# No release date present
stockx_v6.iloc[835]['right'] = 'https://stockx.imgix.net/Air-Force-1-07-White-Black.jpg?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&trim=color&updated_at=1580422905&w=1000'
stockx_v6.iloc[835]['r_d'] = 'https://stockx.imgix.net/Air-Force-1-07-White-Black.jpg?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&trim=color&updated_at=1580422905&w=1000'
stockx_v6.iloc[835]['straight'] = 'https://stockx.imgix.net/Air-Force-1-07-White-Black.jpg?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&trim=color&updated_at=1580422905&w=1000'
stockx_v6.iloc[835]['l_d'] = 'https://stockx.imgix.net/Air-Force-1-07-White-Black.jpg?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&trim=color&updated_at=1580422905&w=1000'
stockx_v6.iloc[835]['left'] = 'https://stockx.imgix.net/Air-Force-1-07-White-Black.jpg?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&trim=color&updated_at=1580422905&w=1000'
stockx_v6.iloc[835]['back'] = 'https://stockx.imgix.net/Air-Force-1-07-White-Black.jpg?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&trim=color&updated_at=1580422905&w=1000'

In [119]:
# This shoe is not currently out which explains the lack of picture
stockx_v6.iloc[157]

stockx_link    https://stockx.com/nike-air-max-270-react-travis-scott
stockx_name                   Nike Air Max 270 React ENG Travis Scott
type_1                                                        Air Max
type_2                                                          Other
type_3                                                        Air Max
model                                                     Air Max 270
right                                                             NaN
r_d                                                               NaN
straight                                                          NaN
l_d                                                               NaN
left                                                              NaN
back                                                              NaN
Style                                                      CT2864-200
Colorway                   Light Cream/Dark Hazel-Mica Green-Starfish
RetailPrice         

In [123]:
# Neither of these shoes have a STYLE category on stock x and the second doesnt have a retail price either
stockx_v6[stockx_v6.Style.isnull()==True]

,stockx_link,stockx_name,type_1,type_2,type_3,model,right,r_d,straight,l_d,left,back,Style,Colorway,RetailPrice,ReleaseDate
134,https://stockx.com/nike-air-max-90-undefeated-white-optic-yellow,Nike Air Max 90 Undefeated White Optic Yellow,Air Max,90,Air Max 90,Air Max 90,"https://stockx-360.imgix.net/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Images/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Lv2/img01.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1578691530","https://stockx-360.imgix.net/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Images/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Lv2/img04.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1578691530","https://stockx-360.imgix.net/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Images/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Lv2/img10.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1578691530","https://stockx-360.imgix.net/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Images/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Lv2/img13.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1578691530","https://stockx-360.imgix.net/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Images/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Lv2/img19.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1578691530","https://stockx-360.imgix.net/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Images/Nike-Air-Max-90-Undefeated-White-Optic-Yellow/Lv2/img28.jpg?auto=format,compress&w=559&q=90&dpr=2&updated_at=1578691530",NaN,White/Opti Yellow,$150,12/30/2019
837,https://stockx.com/nike-react-boot-undercover-brown,Nike React Boot Undercover Brown,Other,Running,Running,React Boot Undercover,"https://stockx.imgix.net/Nike-React-Boot-Undercover-Brown.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1574719849","https://stockx.imgix.net/Nike-React-Boot-Undercover-Brown.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1574719849","https://stockx.imgix.net/Nike-React-Boot-Undercover-Brown.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1574719849","https://stockx.imgix.net/Nike-React-Boot-Undercover-Brown.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1574719849","https://stockx.imgix.net/Nike-React-Boot-Undercover-Brown.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1574719849","https://stockx.imgix.net/Nike-React-Boot-Undercover-Brown.png?fit=fill&bg=FFFFFF&w=700&h=500&auto=format,compress&q=90&dpr=2&trim=color&updated_at=1574719849",NaN,Brown/Black-Teal,NaN,11/30/2019


In [ ]:
# Ultimately the nans remaining won't be detrimental to my data, I will drop the Travis Scott shoe
# as it has no picture there is no need to include it

In [122]:
stockx_v6.isnull().sum()

stockx_link     0
stockx_name     0
type_1          0
type_2          0
type_3          0
model           0
right           1
r_d             1
straight        1
l_d             1
left            1
back            1
Style           2
Colorway        0
RetailPrice    46
ReleaseDate    69
dtype: int64

In [125]:
stockx_v7 = stockx_v6[stockx_v6.right.isnull() == False]

In [126]:
stockx_v7.to_csv('stockx_complete.csv')